In [4]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import numpy as np
from torchvision.utils import save_image

os.chdir('./')

device = 'cuda' if torch.cuda.is_available() else 'cpu'


BATCH_SIZE=64
EPOCHS=50

cond_shape=10
intermediate_dim=400
z_dim=20

torch.manual_seed(42)

train_data = datasets.MNIST(root="../data/", train=True, download=True, transform=transforms.ToTensor(),target_transform=None)
test_data = datasets.MNIST(root="../data/", train=True, download=False, transform=transforms.ToTensor(),target_transform=None)

train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,  drop_last=True, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE,  drop_last=True, shuffle=False)


train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [5]:
flat_img=torch.flatten(train_data[0][0])
flat_shape=list(flat_img.shape)
flat_shape[0]



784

In [6]:
class CVAE(nn.Module):
    def __init__(self):
        super(CVAE, self).__init__()

        self.enc_in = nn.Linear(in_features=flat_shape[0]+cond_shape, out_features=intermediate_dim)
        self.mu = nn.Linear(in_features=intermediate_dim, out_features=z_dim)
        self.logvar = nn.Linear(in_features=intermediate_dim, out_features=z_dim)
        self.dec_in = nn.Linear(in_features=z_dim+cond_shape, out_features=intermediate_dim)
        self.dec_out = nn.Linear(in_features=intermediate_dim, out_features=flat_shape[0]+cond_shape)

    def encoder(self, inp):
        x, y = inp
        y = F.one_hot(y, cond_shape)
        cat = torch.cat((x, y), dim=1)
        # print(cat.shape)
        enc_in_ret = F.relu(self.enc_in(cat))
        # print(enc_in_ret.shape)
        mu_out = F.relu(self.mu(enc_in_ret))
        # print(mu_out.shape)
        logvar_out = F.relu(self.logvar(enc_in_ret))
        # print(logvar_out.shape)
        return mu_out, logvar_out

    def reparametarize(self, mu, logvar):
        sd = torch.exp(0.5*logvar)
        eps = torch.randn_like(sd)
        return mu + eps*sd

    def decoder(self, inp):
        z, y = inp
        y = F.one_hot(y, cond_shape)
        cat = torch.cat((z,y), dim=1)
        dec_in_ret = F.relu(self.dec_in(cat))
        dec_out = torch.sigmoid(self.dec_out(dec_in_ret))
        return dec_out

    def forward(self, inp):
        x, y = inp
        x = x.to(device)
        y = y.to(device)
        mu, logvar = self.encoder((x.view(-1, flat_shape[0]), y))
        z = self.reparametarize(mu, logvar)
        recon = self.decoder((z, y))
        return recon, mu, logvar

def loss_fn(recon, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon, x, reduction='sum')        
    KLD = -0.5*torch.sum(1+logvar-mu.pow(2)-logvar.exp())
    return BCE+KLD

model = CVAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:

def train(epoch):
    
    model.train()
    train_loss = 0


    for batch, (X, y) in enumerate(train_dataloader):
        X = X.to(device) #[64, 1, 28, 28]
        
        
        
        # 1. Forward pass
        recon_batch, mu, logvar = model((X, y))
        
        flat_data = X.view(-1, flat_shape[0]).to(device)                            
        y_onehot = F.one_hot(y, cond_shape).to(device)
        inp = torch.cat((flat_data, y_onehot), 1)
        
        # 2. Calculate loss
        loss = loss_fn(recon_batch, inp, mu, logvar)
        train_loss += loss.item()

        # 3. Zero grad
        optimizer.zero_grad()

        # 4. Backprop
        loss.backward()

        # 5. Step
        optimizer.step()

        if batch % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch * len(X),
                len(train_dataloader.dataset),
                100. * batch / len(train_dataloader),
                loss.item() / len(X)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_dataloader.dataset)))

In [10]:
def test(epoch):
    #Sets the module in evaluation mode
    model.eval()
    test_loss = 0

    with torch.inference_mode():
        for i, (X, y) in enumerate(test_dataloader):
            X = X.to(device)

            # 1. Forward pass
            recon_batch, mu, logvar = model((X, y))
            
            flat_data = X.view(-1, flat_shape[0]).to(device)
            y_onehot = F.one_hot(y, cond_shape).to(device)
            inp = torch.cat((flat_data, y_onehot), 1)

            # 2. Loss
            test_loss += loss_fn(recon_batch, inp, mu, logvar).item()

            # 3. Save images
            if epoch%5==0 and i == 0:
                n = min(X.size(0), 8)
                recon_image = recon_batch[:, 0:recon_batch.shape[1]-10]
                print(recon_image.shape)
                recon_image = recon_image.view(BATCH_SIZE, 1, 28,28)
                print('---',recon_image.shape)
                comparison = torch.cat([X[:n],
                                      recon_image.view(BATCH_SIZE, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_dataloader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)

    # Generate random digits every n epochs
    with torch.inference_mode():
        if epoch%5==0:
            sample = torch.randn(64, 20).to(device)
        
            c = np.zeros(shape=(sample.shape[0],))
            rand = np.random.randint(0, 10)
            print(f"Random number: {rand}")
            c[:] = rand
            c = torch.FloatTensor(c)
            c = c.to(torch.int64)
            c = c.to(device)
            sample = model.decoder((sample, c)).cpu()
            
            generated_image = sample[:, 0:sample.shape[1]-10]
            
            
            save_image(generated_image.view(64, 1, 28, 28),
                    'results/sample_' + str(epoch) + '.png')


Train Epoch: 1 [0/60000 (0%)]	Loss: 555.004333
Train Epoch: 1 [640/60000 (1%)]	Loss: 328.729187
Train Epoch: 1 [1280/60000 (2%)]	Loss: 240.404877
Train Epoch: 1 [1920/60000 (3%)]	Loss: 221.744507
Train Epoch: 1 [2560/60000 (4%)]	Loss: 223.416885
Train Epoch: 1 [3200/60000 (5%)]	Loss: 212.413544
Train Epoch: 1 [3840/60000 (6%)]	Loss: 212.471512
Train Epoch: 1 [4480/60000 (7%)]	Loss: 205.081024
Train Epoch: 1 [5120/60000 (9%)]	Loss: 201.489761
Train Epoch: 1 [5760/60000 (10%)]	Loss: 199.155457
Train Epoch: 1 [6400/60000 (11%)]	Loss: 190.508530
Train Epoch: 1 [7040/60000 (12%)]	Loss: 187.039413
Train Epoch: 1 [7680/60000 (13%)]	Loss: 196.966064
Train Epoch: 1 [8320/60000 (14%)]	Loss: 186.597595
Train Epoch: 1 [8960/60000 (15%)]	Loss: 185.775269
Train Epoch: 1 [9600/60000 (16%)]	Loss: 194.084595
Train Epoch: 1 [10240/60000 (17%)]	Loss: 183.006027
Train Epoch: 1 [10880/60000 (18%)]	Loss: 176.302917
Train Epoch: 1 [11520/60000 (19%)]	Loss: 172.387680
Train Epoch: 1 [12160/60000 (20%)]	Loss: 

In [2]:
sample = torch.randn(64, 20).to(device)
        
c = np.zeros(shape=(sample.shape[0],))
rand = 9
print(f"Random number: {rand}")
c[:] = rand
c = torch.FloatTensor(c)
c = c.to(torch.int64)
c = c.to(device)
sample = model.decoder((sample, c)).cpu()
generated_image = sample[:, 0:sample.shape[1]-10]
            
            
save_image(generated_image.view(64, 1, 28, 28),
        'results/generated_' + str(rand) + '.png')


Random number: 9


NameError: name 'model' is not defined